In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
import utils_zh

_ = load_dotenv(find_dotenv())

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
    )

# langchain是一种快速开发应用程序框架，组件可以链式组合

In [2]:
# 加载向量库，其中包含了所有课程材料的 Embedding。
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'chroma/'
embedding = OpenAIEmbeddings(model='text-embedding-3-small')
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

/var/folders/wq/yyg3qh714970yjrhkxr_q3640000gn/T/ipykernel_63253/721330434.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embedding = OpenAIEmbeddings(model='text-embedding-3-small')


In [3]:
question = "这门课的主要内容是什么？"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [5]:
from langchain.chat_models.openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
llm.predict("Hello world!")

/var/folders/wq/yyg3qh714970yjrhkxr_q3640000gn/T/ipykernel_63253/2851185033.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
/var/folders/wq/yyg3qh714970yjrhkxr_q3640000gn/T/ipykernel_63253/2851185033.py:3: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  llm.predict("Hello world!")


'Hello! How can I assist you today?'

In [6]:
# 构建 prompt
from langchain.prompts import PromptTemplate
template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
有用的回答:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)


In [7]:
# 运行 chain
from langchain.chains import RetrievalQA
question = "概率是课堂主题吗？"
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


result = qa_chain({"query": question})
result["result"]

/var/folders/wq/yyg3qh714970yjrhkxr_q3640000gn/T/ipykernel_63253/1202069427.py:10: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  result = qa_chain({"query": question})


'是的，概率是课堂主题之一，但不会非常深入。谢谢你的提问！'

#### 记忆

In [12]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True#将以消息列表的形式返回聊天记录，而不是单个字符串
)

#### 对话检索链

In [13]:
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
retriever=vectordb.as_retriever()

qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory#
)

In [14]:
question = "概率是一个课堂主题吗？"
result = qa({"question": question})
print(result['answer'])

是的，概率是这门课程的一个主题。在这门课程中，教授假设学生们对基本的概率和统计学有所了解，因此会涉及概率相关的内容。


In [15]:
question = "为什么需要这些先决条件？"
result = qa({"question": question})
print(result['answer'])

这些先决条件的必要性在于确保学生具备足够的概率统计和基本线性代数知识，以便能够理解和学习课程中涉及的概念和技术。这些知识是课程的基础，有助于学生更好地理解课程内容并顺利完成学习任务。


#### 创建聊天机器人

In [16]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.chat_models.openai import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

In [17]:
def load_db(file, chain_type, k):
    """
    该函数用于加载 PDF 文件，切分文档，生成文档的嵌入向量，创建向量数据库，定义检索器，并创建聊天机器人实例。

    参数:
    file (str): 要加载的 PDF 文件路径。
    chain_type (str): 链类型，用于指定聊天机器人的类型。
    k (int): 在检索过程中，返回最相似的 k 个结果。

    返回:
    qa (ConversationalRetrievalChain): 创建的聊天机器人实例。
    """
    # 载入文档
    loader = PyPDFLoader(file)
    documents = loader.load()
    # 切分文档
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    # 定义 Embeddings
    embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
    # 根据数据创建向量数据库
    db = DocArrayInMemorySearch.from_documents(docs, embeddings)
    # 定义检索器
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # 创建 chatbot 链，Memory 由外部管理
    qa = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model_name=llm_name, temperature=0), 
        chain_type=chain_type, 
        retriever=retriever, 
        return_source_documents=True,
        return_generated_question=True,
    )
    return qa 

In [ ]:
######稍等